In [40]:
import time
import random
import os

from selenium import webdriver
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import json

import requests

import import_ipynb

ModuleNotFoundError: No module named 'import_ipynb'

In [2]:
_debug = False

In [3]:
uploaded_posters_ids = []
all_posters = {}
all_posters['posters'] = []

## Open Session

In [4]:
# open indico
driver = webdriver.Firefox()
driver.get("https://indico.fnal.gov/event/19348/manage/contributions/") # url for our conference
# assert "INDICO" in driver.title

#login
sso_btn = driver.find_element_by_class_name('external-provider-shib-sso')
sso_btn.click()

continue_btn = driver.find_element_by_class_name('ping-button')
continue_btn.click()

inputUser = driver.find_element_by_id('username')
inputUser.clear()
inputUser.send_keys("mdeltutt")

inputPassword = driver.find_element_by_id('password')
inputPassword.clear()
# inputPassword.send_keys(os.environ('SSO_PWD'))
inputPassword.send_keys('3Razumichin')
inputPassword.send_keys(Keys.RETURN)

In [36]:
%%bash -c : # This prevents cell from being executed

poster_table = driver.find_element_by_css_selector('table')
poster_table_lines = poster_table.find_elements_by_css_selector('tr')
# postertable[20].get_attribute('innerHTML')
# postertable[3].find_elements_by_tag_name('td')[3].get_attribute('innerHTML')
print('Number of posters', len(poster_table_lines))

poster_table_lines[1].find_elements_by_tag_name('td')[3].get_attribute('innerHTML')

cell = poster_table_lines[568].find_elements_by_tag_name('td')[6]
print(cell.get_attribute('innerHTML'))
cell.find_element_by_class_name('person-row').get_attribute('innerHTML')

cell = poster_table_lines[568].find_elements_by_tag_name('td')[11]
print(cell.get_attribute('innerHTML'))
material = cell.find_element_by_class_name('icon-attachment')
material.click()

In [37]:
%%bash -c : # This prevents cell from being executed
material_table = driver.find_element_by_class_name('tree')
material_table_lines = material_table.find_elements_by_css_selector('tr')
# postertable[20].get_attribute('innerHTML')
print('Number of materials', len(material_table_lines))
for row in material_table_lines:
    # The file name is the zero index td cell:
    cell = row.find_elements_by_tag_name('td')[0]
    print(cell.text)
    if '.pdf' not in cell.text:
        print('Not a PDF file!')
    material_link = cell.find_element_by_css_selector("[href]").get_attribute('href')
    print(material_link)
btn = driver.find_elements_by_class_name('ui-dialog-titlebar-close')
btn[0].click()

In [23]:
def get_materials_link():
    '''
    With a material pop-up window opened,
    this function reads all the materials uploaded
    and picks the first PDF file found.
    
    returns:
    - file name
    - link to file
    '''
    material_table = driver.find_element_by_class_name('tree')
    material_table_lines = material_table.find_elements_by_css_selector('tr')
    
    material_link = 'NotFound'
    file_name = 'NotFound'

    print('Number of materials', len(material_table_lines))
    for row in material_table_lines:
        # The file name is the zero index td cell:
        cell = row.find_elements_by_tag_name('td')[0]
#         print(cell.text)
        if '.pdf' in cell.text:
            file_name = cell.text
            material_link = cell.find_element_by_css_selector("[href]").get_attribute('href')
#             print(material_link)
            break
    return file_name, material_link

In [24]:
poster_table = driver.find_element_by_css_selector('table')
poster_table_lines = poster_table.find_elements_by_css_selector('tr')

Loop over all the contribution lines, and for each of them get:
- (element 6): the author name
- (element 11): the link to the poster

In [47]:
for i, row in enumerate(poster_table_lines):
    author = 'None'
    link = 'None'
    file_name = 'None'
    
    for j, cell in enumerate(row.find_elements_by_tag_name('td')):
#         print(cell.text)
        if j == 6:
            try:
                author = cell.find_element_by_class_name('person-row').get_attribute('innerHTML')
            except:
                author = 'None'
                print('WARNING: Cannot find author name for')
            print('Author:', author)
            
        if j == 11:
            material = cell.find_element_by_class_name('icon-attachment')
            if material.text == 'None':
                link = 'None'
            else:
                material.click()
                file_name, link = get_materials_link()
                close_btn = driver.find_elements_by_class_name('ui-dialog-titlebar-close')
                close_btn[0].click()
                
            print('Link:', link)
            
    if author is not 'None':
        all_posters['posters'].append({'author': author,
                                       'file_name': file_name,
                                       'file_link': link
                                      })

    if (i > 10 and _debug):
        break

Author: Avinay Bhat
Link: None
Author: ITISHREE SETHI
Link: None
Author: Marcos Dracos
Link: None
Author: Nick Solomey
Link: None
Author: Huiling Li
Link: None
Author: James Kneller
Link: None
Author: Peibo An
Link: None
Author: Chris Rogers
Link: None
Author: Tom Lord
Link: None
Author: Paul Jurj
Link: None
Author: Craig Brown
Link: None
Author: Cedric Cerna
Link: None
Author: Tamer Tolba
Link: None
Author: Jan Behrens
Link: None
Author: Lukas Hauertmann
Link: None
Author: Xiang Liu
Link: None
Author: Martin Schuster
Link: None
Author: Osamu Yasuda
Link: None
Author: Jianming Bian
Link: None
Author: Yu-Feng Li
Link: None
Author: Sankagiri Umasankar
Link: None
Author: Wojciech Flieger
Link: None
Author: Justin Evans
Link: None
Author: Jaydeep Datta
Link: None
Author: Florian Priester
Link: None
Author: Mark Anderson
Link: None
Author: Daljeet Kaur
Link: None
Author: José SOTO
Link: None
Author: Ana Gallego Ros
Link: None
Author: Linyan WAN
Link: None
Author: Matheus Hostert
Link: None


Link: None
Author: Daniel Siegmann
Link: None
Author: Xingyu Zhao
Link: None
Author: Andrey Romanov
Link: None
Author: Maxim Gromov
Link: None
Author: Ushak Rahaman
Link: None
Author: Alessio Giarnetti
Link: None
Author: J. Pedro Ochoa
Link: None
Author: Manoj Kumar Singh
Link: None
Author: Elizabeth Friedman
Link: None
Author: Michael Larson
Link: None
Author: Shreyashi Chakdar
Link: None
Author: Ran Itay
Link: None
Author: Davio Cianci
Link: None
Author: Michael Willers
Link: None
Author: Mauricio Bustamante
Link: None
Author: Joseph Smolsky
Link: None
Author: Irina Lomskaya
Link: None
Author: Deepak Tiwari
Link: None
Author: Brenda Aurea Cervantes Vergara
Link: None
Author: yaping cheng
Link: None
Author: Hiroshi Ogawa
Link: None
Author: Lu Ren
Link: None
Author: Damini Singh
Link: None
Author: Laura Zambelli
Link: None
Author: Guido Fantini
Link: None
Author: Matteo Cerruti
Link: None
Author: Aldo Ianni
Link: None
Author: Etienne Chardonnet
Link: None
Author: Jinyu Kim
Link: None
A

Author: Daniel Smith
Link: None
Author: Casandra Morris
Link: None
Author: Ka Vang Tsang
Link: None
Author: Gabriela Vitti Stenico
Link: None
Author: Doga Veske
Link: None
Author: Russell Neilson
Link: None
Author: Benjamin Foust
Link: None
Author: Jarrett Moon
Link: None
Author: Talia Weiss
Link: None
Author: Eliza Gazda
Link: None
Author: Karla Tellez-Giron-Flores
Link: None
Author: Dario Rodrigues
Link: None
Author: Victor Buridon
Link: None
Author: Ivan Pedro Sidelnik
Link: None
Author: Bryan Ramson
Link: None
Author: Vivek Singh
Link: None
Author: Tim Daniels
Link: None
Author: XIANYI ZHANG
Link: None
Author: Tyler Rehak
Link: None
Author: Marjon Moulai
Link: None
Author: Behzad Hosseini
Link: None
Author: Olivia Dalager
Link: None
Author: Ankur Nath
Link: None
Author: Lorna Nolan
Link: None
Author: David Henaff
Link: None
Author: Deepika Jena
Link: None
Author: BENITO VARGAS PEREZ
Link: None
Author: Jaret Heise
Link: None
Author: Ryan Dorrill
Link: None
Author: Pranav Dave
Link: 

ValueError: too many values to unpack (expected 2)

In [44]:
with open('posters.json', 'w') as outfile:
    json.dump(all_posters, outfile, indent=4)

## Download all posters

In [45]:
poster_dir = 'posters/'
os.system(f'mkdir -p {poster_dir}')

counter = 0

for p in all_posters['posters']:
    link = p['file_link']
    os.system(f'curl -o {poster_dir}/poster_{counter} {link}')

KeyError: 'file_link'

# Original

In [75]:
# open zenodo
driver = webdriver.Chrome()
driver.get("https://urldefense.proofpoint.com/v2/url?u=https-3A__zenodo.org_deposit_new-3Fc-3Dneutrino2018-255C&d=DwIFaQ&c=gRgGjJ3BkIsb5y6s49QqsA&r=OM-vxifKHcXuA7j15dia9pvU4aZnfF-vEDdjO1w0uAo&m=zKbEEq5BNyV_8ft4dySXrbChhrkdRSIOuU34TcDXGEY&s=6y_Uj97O3ptHTSe65MTTaqFmnTwCNpYEGxueUysH9hI&e= ") # url for our conference
assert "Zenodo" in driver.title

#login
inputUser = driver.find_element_by_id('email')
inputUser.clear()
inputUser.send_keys("email") #change to your email

inputPassword = driver.find_element_by_id('password')
inputPassword.clear()
inputPassword.send_keys("insertpasswordhere") # nb, the password is in plain letters
inputPassword.send_keys(Keys.RETURN)

# open indico (in another tab)
driver.execute_script("window.open('https://indico.desy.de/indico/event/18342/manage/contributions/');%5C") # change to your url for indico
driver.switch_to.window(driver.window_handles[1])
assert "Neutrino" in driver.title

#login
inputUser = driver.find_element_by_id('usernameInput').find_elements_by_css_selector("*")[0]
inputUser.clear()
inputUser.send_keys("user")

inputPassword = driver.find_element_by_id('passwordInput').find_elements_by_css_selector("*")[0]
inputPassword.clear()
inputPassword.send_keys("insertpasswordhere") #another password in clear letters
inputPassword.send_keys(Keys.RETURN)

# Save the window opener (current window, do not mistaken with tab... not the same)
main_window = driver.current_window_handle

In [76]:
# save posters table
postertable = driver.find_element_by_id('contribsItems').find_elements_by_css_selector("*")
print(len(postertable))

6330


In [77]:
index = 0
# to find the name, it is necessary to count the lines from the top. This index stores the position of the first line of each row
index_top = 0

# this is a local file that stores the ids of posts that have already been uploaded
uploadposterfile = "uploaded_posters.txt" 

while index < 0:
    # focus on the first tab
    driver.switch_to.window(driver.window_handles[1])
    row = postertable[index]
    row_html = row.get_attribute('innerHTML')
    if 'slides' in row_html and '<td' in row_html: #the whole thing
        index_top = index
    if 'slides' in row.get_attribute('innerHTML') and '<td' not in row.get_attribute('innerHTML'): #only the poster
        print(index-index_top)
        session_number = postertable[index_top+10].get_attribute('innerHTML')
        id_number = postertable[index_top+3].get_attribute('innerHTML')
        if session_number == "":
            print(session_number, id_number)
            session_number = postertable[index_top+11].get_attribute('innerHTML')
            
        try:
            file = open(uploadposterfile,"r")
            if int(id_number) in uploaded_posters_ids or id_number in file.read():
                index += 1
                print("Poster with id " + id_number + " already uploaded")
                continue
            file.close()
        except FileNotFoundError:
            if int(id_number) in uploaded_posters_ids:
                index += 1
                print("Poster with id " + id_number + " already uploaded")
                continue
        
        # open a new tab with poster infos
        url = postertable[index_top+7].get_attribute('innerHTML')
        url = url.split('"')
        url_js = "window.open('" + (url[1]) + "');"
        print(url_js)
        driver.execute_script(url_js)
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[2])
        
        # check if the poster should be included in the procedings
        postertext = driver.find_element_by_xpath("//*[contains(text(), 'Poster included in proceedings')]")
        posterparent = postertext.find_element_by_xpath("..")
        posterbool = posterparent.find_element_by_xpath("..")
        posterboolstring = posterbool.get_attribute('innerHTML')
        if 'yes' in posterboolstring:
            posterbool = True
        else:
            posterbool = False
        
        # in case yes, save the abstract
        if posterbool:
            webtext = driver.find_element_by_xpath("//*[contains(text(), 'Abstract content')]")
            parent_1 = webtext.find_element_by_xpath("..")
            parent_2 = parent_1.find_element_by_xpath("..")
            #print(parent_2.get_attribute('innerHTML'))
            
            # old way to save the title, unused because of formatting issues
            """title = driver.find_element_by_class_name('blacktext').get_attribute('innerHTML').replace('<b>', '')
            title = title.replace('</b>', '')
            title = title.replace('&nbsp;', ' ')"""
            abstractlist = driver.find_elements_by_class_name('md-preview-wrapper')
            abstract = abstractlist[2].get_attribute('innerHTML')#.replace('<p>', "")
            #abstract = abstract.replace('</p>', "")
            names = driver.find_elements_by_class_name('authorMove')
            name = names[-1].get_attribute('innerHTML')
            name = name.split('(')
            fullname = name[0]
            name[1] = name[1].split(')')
            institution = name[1][0]
            print(title, fullname, institution)
        # go back to the main window
        driver.close()
        driver.switch_to.window(driver.window_handles[1])
        
        # if the poster needs to be downloaded...
        if posterbool:
            title = postertable[index_top+8].get_attribute('innerHTML')
            title = title.replace('&nbsp;', ' ')
            
            for i in range(5):
                # compose poster url from session number and id. The number at the end of the url can increase
                poster_url = 'https://indico.desy.de/indico/event/18342/session/' + str(session_number) + '/contribution/' + str(id_number) + '/material/slides/' + str(i) + '.pdf'
                print(poster_url)
            
                # save pdf with requests
                response = requests.get(poster_url)
                with open('poster.pdf', 'wb') as f:
                    f.write(response.content)
                
                try:
                    doc = PdfFileReader(open("poster.pdf", "rb"))
                    npages = doc.getNumPages()
                    if npages == 1:
                        poster_good = True
                    else:
                        print('Poster with id ' + id_number + ' are actually slides')
                        poster_good = False
                    break
                except:
                    if i<4:
                        pass
                    else:
                        print('Poster with id ' + id_number + ' not found')
                        file2 = open("u_not_found_posters.txt","a")
                        file2.write(id_number + '\n')
                        file2.close()
                        poster_good = False
            # upload poster function in next cell
            if poster_good:
                #waitinput = input('Press any key to continue')
                upload_poster(title, fullname, institution, abstract)             
                print("called upload function for poster with id " + id_number)
            
            # save on external txt file the posters uploaded (we need to be extra careful with double entries)
            file1 = open(uploadposterfile,"a")
            file1.write(id_number + '\n')
            file1.close()
    index += 1

13
Poster with id 94 already uploaded
14
Poster with id 94 already uploaded
13
Poster with id 95 already uploaded
14
Poster with id 95 already uploaded
13
window.open('https://indico.desy.de/indico/event/18342/manage/session/35/contribution/101/');%5CnSearching for Beyond the Standard Model Physics with the DUNE Experiment LI, Yufeng  Institute of High Energy Physics
https://indico.desy.de/indico/event/18342/session/35/contribution/101/material/slides/0.pdf%5Cncalled upload function for poster with id 101
14
Poster with id 101 already uploaded
13
window.open('https://indico.desy.de/indico/event/18342/manage/session/35/contribution/105/');%5CnAverage CsI neutron density distribution from COHERENT data SUEMATSU, Daijiro  Kanazawa University
https://indico.desy.de/indico/event/18342/session/35/contribution/105/material/slides/0.pdf%5Cncalled upload function for poster with id 105
14
Poster with id 105 already uploaded
13
Poster with id 108 already uploaded
14
Poster with id 108 already up

called upload function for poster with id 155
14
Poster with id 155 already uploaded
13
Poster with id 156 already uploaded
14
Poster with id 156 already uploaded
13
Poster with id 158 already uploaded
14
Poster with id 158 already uploaded
13
Poster with id 161 already uploaded
14
Poster with id 161 already uploaded
14
 164
Poster with id 164 already uploaded
15
 164
Poster with id 164 already uploaded
14
 165
Poster with id 165 already uploaded
15
 165
Poster with id 165 already uploaded
13
Poster with id 166 already uploaded
14
Poster with id 166 already uploaded
13
Poster with id 169 already uploaded
14
Poster with id 169 already uploaded
13
Poster with id 173 already uploaded
14
Poster with id 173 already uploaded
13
Poster with id 188 already uploaded
14
Poster with id 188 already uploaded
15
 189
Poster with id 189 already uploaded
16
 189
Poster with id 189 already uploaded
13
window.open('https://indico.desy.de/indico/event/18342/manage/session/34/contribution/190/');%5CnPoten

13
Poster with id 268 already uploaded
14
Poster with id 268 already uploaded
13
Poster with id 269 already uploaded
14
Poster with id 269 already uploaded
13
Poster with id 271 already uploaded
14
Poster with id 271 already uploaded
13
Poster with id 276 already uploaded
14
Poster with id 276 already uploaded
13
Poster with id 278 already uploaded
14
Poster with id 278 already uploaded
13
Poster with id 279 already uploaded
14
Poster with id 279 already uploaded
13
Poster with id 280 already uploaded
14
Poster with id 280 already uploaded
14
 283
Poster with id 283 already uploaded
15
 283
Poster with id 283 already uploaded
13
Poster with id 284 already uploaded
14
Poster with id 284 already uploaded
13
Poster with id 286 already uploaded
14
Poster with id 286 already uploaded
13
Poster with id 287 already uploaded
14
Poster with id 287 already uploaded
14
 288
Poster with id 288 already uploaded
15
 288
Poster with id 288 already uploaded
13
Poster with id 290 already uploaded
14
Po

13
Poster with id 364 already uploaded
14
Poster with id 364 already uploaded
13
Poster with id 367 already uploaded
14
Poster with id 367 already uploaded
13
Poster with id 370 already uploaded
14
Poster with id 370 already uploaded
13
Poster with id 372 already uploaded
14
Poster with id 372 already uploaded
13
Poster with id 380 already uploaded
14
Poster with id 380 already uploaded
13
Poster with id 382 already uploaded
14
Poster with id 382 already uploaded
13
window.open('https://indico.desy.de/indico/event/18342/manage/session/34/contribution/386/');%5CnModeling of the response function of the KATRIN experiment GREEN, Matthew  North Carolina State University / Oak Ridge National Laboratory
https://indico.desy.de/indico/event/18342/session/34/contribution/386/material/slides/0.pdf%5Cncalled upload function for poster with id 386
14
Poster with id 386 already uploaded
13
Poster with id 387 already uploaded
14
Poster with id 387 already uploaded
13
Poster with id 388 already uploa

In [52]:
def upload_poster(title, name, institution, abstract):
    # fill in various infos
    driver.switch_to.window(driver.window_handles[0])
    # save abstract editor for writing in it later
    time.sleep(1)
    editorFrame = driver.find_element_by_xpath("/html/body/div/div/invenio-files-uploader/invenio-records/invenio-records-form/form/bootstrap-decorator[3]/fieldset/div/div[2]/sf-decorator[6]/div/sf-decorator/div/div/div/div/div/iframe")

    driver.find_element_by_xpath('/html/body/div/div/invenio-files-uploader/invenio-records/invenio-records-form/form/bootstrap-decorator[2]/fieldset/div/div[2]/sf-decorator[1]/div/ul/li[2]/label').click()
    webinput = driver.find_element_by_id('title')
    webinput.clear()
    webinput.send_keys(title)
    webinput = driver.find_element_by_id('name')
    webinput.clear()
    webinput.send_keys(name)
    webinput = driver.find_element_by_id('affiliation')
    webinput.clear()
    webinput.send_keys(institution)
    
    # write the abstract
    # editor is in another frame
    driver.switch_to.frame(editorFrame)
    bodyAbstract = driver.find_element_by_tag_name("body")
    bodyAbstract.clear()
    raw_abstract = "%r"%abstract.strip()
    abstractscript = "arguments[0].innerHTML = " + raw_abstract
    # use js to change the inner html of the editor
    driver.execute_script(abstractscript, bodyAbstract)
    
    driver.switch_to.window(driver.window_handles[0])
    webinput = driver.find_element_by_id('orcid')
    webinput.clear()
    webinput.send_keys('\t')
    
    # actually upload the poster
    uploadbutton = driver.find_element_by_xpath('//*[@id="invenio-records"]/invenio-files-uploader/invenio-records/invenio-files-list/div[1]/div[2]/div/div/h1/button')
    uploadbutton.click()
    
    # MAC OS X ONLY PART:
    # hardcoding the mouse position to click the upload button from mac os x
    width, height = autopy.screen.size()
    autopy.mouse.move(width/2, height*0.26)
    time.sleep(3)
    autopy.mouse.click()
    autopy.mouse.toggle(down=True)
    time.sleep(0.5)
    autopy.mouse.toggle(down=False)
    autopy.mouse.move(944.21484375, 547.8984375)
    time.sleep(0.5)
    autopy.mouse.click()
    # angry clicking until it works
    angrykid = 4
    for i in range(angrykid):
        time.sleep(0.5)
        autopy.mouse.toggle(down=True)
        time.sleep(0.5)
        autopy.mouse.toggle(down=False)
    # I'm going to regret so much I didn't find a better way...
    time.sleep(1)
    # uploading
    driver.find_element_by_xpath('/html/body/div/div/invenio-files-uploader/invenio-records/invenio-files-list/div[1]/div[1]/div/div[2]/span/button[3]').click()
    
    # waiting for the upload to be finished (it looks for the check)
    time.sleep(20)
    # click save button
    driver.find_element_by_xpath('//*[@id="invenio-records"]/invenio-files-uploader/invenio-records/invenio-records-actions[1]/div[1]/ul/li[4]/button').click()
    # click publish button 
    driver.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="invenio-records"]/invenio-files-uploader/invenio-records/invenio-records-actions[1]/div[1]/ul/li[5]/button').click()
    # click "I understand" popup
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="warningModal"]/div/div/div[3]/button[2]').click()
    # go back to upload page
    driver.get("https://urldefense.proofpoint.com/v2/url?u=https-3A__zenodo.org_deposit_new-3Fc-3Dneutrino2018-255C&d=DwIFaQ&c=gRgGjJ3BkIsb5y6s49QqsA&r=OM-vxifKHcXuA7j15dia9pvU4aZnfF-vEDdjO1w0uAo&m=zKbEEq5BNyV_8ft4dySXrbChhrkdRSIOuU34TcDXGEY&s=6y_Uj97O3ptHTSe65MTTaqFmnTwCNpYEGxueUysH9hI&e= ")

In [122]:
# find the mouse coordinates for autopy
time.sleep(5)
mouseloc = autopy.mouse.location()
print(mouseloc)

(944.21484375, 547.8984375)
